# 🏠 부동산 실거래가 예측 대회 - KKH - 외부 데이터 추가
> - 학습 데이터에 존재하는 부동산 관련 결측치 정보를 외부 데이터를 활용해 채워 넣는다.
> - 서울시 공동주택 아파트 정보를 활용한다.
> - [서울시 공동주택 아파트 정보](https://data.seoul.go.kr/dataList/OA-15818/S/1/datasetView.do)
> - kimkihong / helpotcreator@gmail.com / Upstage AI Lab 3기
> - 2024.07.16.화 ~ 2024.07.19.금 19:00

## 라이브러리 & 폰트 설정

- 폰트는 .otf 파일을 직접 위치시켜서 임포트 시켰다.
- 본인은 우분투와 윈도우 두 환경에서 동시에 작업 중인데, 이와 같이 폰트를 설정하면, 문제 없다.

In [78]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(fname=r'font/NanumGothic.otf', name='NanumBarunGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'})
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance


# 모든 열을 표시하도록 설정
pd.set_option('display.max_columns', None)

In [79]:
# train = pd.read_csv('data/train.csv', encoding='utf-8')
test = pd.read_csv('data/test.csv', encoding='utf-8')
solution = pd.read_csv('data/solution.csv', encoding='utf-8')
gps = pd.read_csv('data/jaemyung_newXY_for_test.csv', encoding='utf-8')
add = pd.read_csv('data/add.csv', encoding='EUC-KR') # 외부 데이터: train의 결측치를 채워넣어줄 용도임

In [80]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9272 entries, 0 to 9271
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   시군구                     9272 non-null   object 
 1   번지                      9270 non-null   object 
 2   본번                      9272 non-null   float64
 3   부번                      9272 non-null   float64
 4   아파트명                    9262 non-null   object 
 5   전용면적(㎡)                 9272 non-null   float64
 6   계약년월                    9272 non-null   int64  
 7   계약일                     9272 non-null   int64  
 8   층                       9272 non-null   int64  
 9   건축년도                    9272 non-null   int64  
 10  도로명                     9272 non-null   object 
 11  해제사유발생일                 212 non-null    float64
 12  등기신청일자                  9272 non-null   object 
 13  거래유형                    9272 non-null   object 
 14  중개사소재지                  9272 non-null   

In [81]:
# test = test[['시군구', '번지', '아파트명', '전용면적(㎡)', '계약년월', '층', '건축년도', '도로명', '등기신청일자', '좌표X', '좌표Y']]

In [82]:
test.isna().mean() * 100

시군구                        0.000000
번지                         0.021570
본번                         0.000000
부번                         0.000000
아파트명                       0.107852
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   97.713546
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        70.987921
k-전화번호                    70.923210
k-팩스번호                    71.246764
단지소개기존clob                94.025022
k-세대타입(분양형태)              70.772217
k-관리방식                    70.772217
k-복도유형                    70.793788
k-난방방식                    70.772217
k-전체동수                    70.933995
k-전체세대수                   70.772217
k-건설사(시공사)                70.955565
k-시행사                     70.966350
k-사용검사일-사용승인일             70

In [83]:
test[test['아파트명'].isna()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
2451,서울특별시 구로구 구로동,740-7,740.0,7.0,NaN,35.10,202308,21,4,1996,구로동로12길 49,NaN,,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2452,서울특별시 구로구 구로동,743-27,743.0,27.0,NaN,59.56,202309,8,1,1994,구로동로22길 52-2,NaN,20230915,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,서울특별시 구로구 구로동,747-34,747.0,34.0,NaN,49.11,202308,4,2,2000,도림로3길 35-5,NaN,20230828,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2454,서울특별시 구로구 구로동,752-17,752.0,17.0,NaN,33.56,202307,3,3,1994,구로동로22길 76-6,NaN,20230727,중개거래,서울 구로구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2455,서울특별시 구로구 구로동,780-86,780.0,86.0,NaN,35.55,202307,5,5,1961,도림로12길 11,NaN,20230706,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4035,서울특별시 동대문구 장안동,404-13,404.0,13.0,NaN,84.86,202308,18,7,2005,천호대로77길 62,NaN,20230822,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4449,서울특별시 동작구 상도동,323-4,323.0,4.0,NaN,106.51,202307,5,3,2005,국사봉1길 18,NaN,,중개거래,서울 동작구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8041,서울특별시 영등포구 대림동,1101-1,1101.0,1.0,NaN,14.46,202307,8,6,2012,도림천로19길 12,NaN,20230809,중개거래,서울 영등포구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8042,서울특별시 영등포구 대림동,1101-1,1101.0,1.0,NaN,14.46,202307,21,10,2012,도림천로19길 12,NaN,,중개거래,서울 영등포구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8966,서울특별시 중구 신당동,432-904,432.0,904.0,NaN,59.16,202309,13,4,2001,동호로11마길 20-8,NaN,,중개거래,서울 중구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
test.loc[(test['도로명'] == '구로동로12길 49') & (test['층'] == 4) & (test['아파트명'].isna()), '아파트명'] = '(740-7)'
test.loc[(test['도로명'] == '구로동로22길 52-2') & (test['층'] == 1) & (test['아파트명'].isna()), '아파트명'] = '(743-27)'
test.loc[(test['도로명'] == '도림로3길 35-5') & (test['층'] == 2) & (test['아파트명'].isna()), '아파트명'] = '(747-34)'
test.loc[(test['도로명'] == '구로동로22길 76-6') & (test['층'] == 3) & (test['아파트명'].isna()), '아파트명'] = '(752-17)'
test.loc[(test['도로명'] == '도림로12길 11') & (test['층'] == 5) & (test['아파트명'].isna()), '아파트명'] = '(780-86)'
test.loc[(test['도로명'] == '천호대로77길 62') & (test['층'] == 7) & (test['아파트명'].isna()), '아파트명'] = '(아이엔에스새터)'
test.loc[(test['도로명'] == '국사봉1길 18') & (test['층'] == 3) & (test['아파트명'].isna()), '아파트명'] = '상진아파트'
test.loc[(test['도로명'] == '도림천로19길 12') & (test['층'] == 6) & (test['아파트명'].isna()), '아파트명'] = '(1101-1)'
test.loc[(test['도로명'] == '도림천로19길 12') & (test['층'] == 10) & (test['아파트명'].isna()), '아파트명'] = '(1101-1)'
test.loc[(test['도로명'] == '동호로11마길 20-8') & (test['층'] == 4) & (test['아파트명'].isna()), '아파트명'] = '(432-904)'

In [85]:
test[test['아파트명'].str.len() == 1]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [86]:
test[test['번지'].isna()]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
4963,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,19,12,2015,헌릉로8길 45,NaN,,중개거래,서울 서초구,아파트,0220579560,0220579562,NaN,기타,위탁관리,복도식,개별난방,13.0,1077.0,고려개발,에스에치공사,2015-06-18 00:00:00.0,134431.0,59281.0,134431.0,930.0,147.0,0.0,NaN,NaN,NaN,2023-09-23 16:53:27.0,NaN,위탁,단일계약,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0
4964,서울특별시 서초구 내곡동,NaN,0.0,0.0,서초포레스타2단지,84.48,202308,22,11,2015,헌릉로8길 45,NaN,,중개거래,서울 서초구,아파트,0220579560,0220579562,NaN,기타,위탁관리,복도식,개별난방,13.0,1077.0,고려개발,에스에치공사,2015-06-18 00:00:00.0,134431.0,59281.0,134431.0,930.0,147.0,0.0,NaN,NaN,NaN,2023-09-23 16:53:27.0,NaN,위탁,단일계약,위탁,8252.0,1185.0,의무,2019-04-24 15:11:04.0,Y,N,127.062596,37.454703,2015-07-17 11:07:27.0


In [87]:
test.loc[(test['도로명'] == '헌릉로8길 45') & (test['층'] == 12) & (test['번지'].isna()), '번지'] = '384'
test.loc[(test['도로명'] == '헌릉로8길 45') & (test['층'] == 11) & (test['번지'].isna()), '번지'] = '384'

In [88]:
test[test['전용면적(㎡)'] < 10.0]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [89]:
test[test['계약년월'] < 202301]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [90]:
test.isna().mean() * 100

시군구                        0.000000
번지                         0.000000
본번                         0.000000
부번                         0.000000
아파트명                       0.000000
전용면적(㎡)                    0.000000
계약년월                       0.000000
계약일                        0.000000
층                          0.000000
건축년도                       0.000000
도로명                        0.000000
해제사유발생일                   97.713546
등기신청일자                     0.000000
거래유형                       0.000000
중개사소재지                     0.000000
k-단지분류(아파트,주상복합등등)        70.987921
k-전화번호                    70.923210
k-팩스번호                    71.246764
단지소개기존clob                94.025022
k-세대타입(분양형태)              70.772217
k-관리방식                    70.772217
k-복도유형                    70.793788
k-난방방식                    70.772217
k-전체동수                    70.933995
k-전체세대수                   70.772217
k-건설사(시공사)                70.955565
k-시행사                     70.966350
k-사용검사일-사용승인일             70

In [91]:
solution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11275 entries, 0 to 11274
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NO        11275 non-null  int64  
 1   시군구       11275 non-null  object 
 2   번지        11275 non-null  object 
 3   본번        11275 non-null  int64  
 4   부번        11275 non-null  int64  
 5   단지명       11275 non-null  object 
 6   전용면적(㎡)   11275 non-null  float64
 7   계약년월      11275 non-null  int64  
 8   계약일       11275 non-null  int64  
 9   거래금액(만원)  11275 non-null  object 
 10  동         11275 non-null  object 
 11  층         11275 non-null  int64  
 12  매수자       11275 non-null  object 
 13  매도자       11275 non-null  object 
 14  건축년도      11275 non-null  int64  
 15  도로명       11275 non-null  object 
 16  해제사유발생일   11275 non-null  object 
 17  거래유형      11275 non-null  object 
 18  중개사소재지    11275 non-null  object 
 19  등기일자      11275 non-null  object 
dtypes: float64(1), int64(7), obj

In [92]:
test[~test['도로명'].isin(solution['도로명'])]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
2329,서울특별시 광진구 자양동,643-23,643.0,23.0,현대빌라트,131.85,202308,25,6,1993,뚝섬로52마길 42-18,NaN,,중개거래,서울 광진구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2951,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,79.25,202307,18,1,1988,한글비석로 480,20230816.0,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2952,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,79.25,202307,18,1,1988,한글비석로 480,NaN,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2953,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,68.99,202307,22,13,1988,한글비석로 480,NaN,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2954,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,79.25,202308,15,11,1988,한글비석로 480,NaN,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2955,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,68.99,202308,17,11,1988,한글비석로 480,NaN,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2956,서울특별시 노원구 상계동,639,639.0,0.0,보람2단지,68.99,202308,31,10,1988,한글비석로 480,NaN,,중개거래,서울 노원구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
test[~test['아파트명'].isin(solution['단지명'])]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,언주로 3,NaN,,직거래,-,아파트,025776611,025776673,NaN,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,NaN,NaN,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,NaN,직영,단일계약,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.057210,37.476763,2022-11-17 10:19:06.0
36,서울특별시 강남구 대치동,891-6,891.0,6.0,테헤란로대우아이빌,59.83,202309,9,9,2004,테헤란로 428,NaN,,중개거래,서울 강남구,주상복합,025626736,025626737,NaN,분양,위탁관리,복도식,개별난방,1.0,372.0,대우건설,프리즘(주),2004-04-29 00:00:00.0,24850.0,15149.0,19812.0,344.0,28.0,0.0,NaN,NaN,NaN,2023-09-23 17:47:31.0,NaN,위탁,종합계약,위탁,0.0,0.0,의무,2013-06-23 14:20:40.0,Y,N,127.054022,37.505690,2013-03-07 09:47:03.0
59,서울특별시 강남구 도곡동,934-10,934.0,10.0,도곡우성,84.83,202308,7,4,1986,남부순환로363길 49,NaN,,중개거래,"서울 강남구, 서울 성동구",아파트,0234623466,0262413469,NaN,분양,자치관리,혼합식,지역난방,2.0,390.0,우성건설,을지조합,1986-10-31 00:00:00.0,40311.0,32577.0,39792.0,0.0,390.0,0.0,NaN,NaN,NaN,2023-09-20 21:36:10.0,22082605540,직영,종합계약,위탁,0.0,330.0,의무,2013-06-17 13:35:11.0,Y,N,127.037124,37.488203,2013-03-07 09:46:46.0
60,서울특별시 강남구 도곡동,934-10,934.0,10.0,도곡우성,84.83,202308,7,4,1986,남부순환로363길 49,20230830.0,,중개거래,"서울 강남구, 서울 성동구",아파트,0234623466,0262413469,NaN,분양,자치관리,혼합식,지역난방,2.0,390.0,우성건설,을지조합,1986-10-31 00:00:00.0,40311.0,32577.0,39792.0,0.0,390.0,0.0,NaN,NaN,NaN,2023-09-20 21:36:10.0,22082605540,직영,종합계약,위탁,0.0,330.0,의무,2013-06-17 13:35:11.0,Y,N,127.037124,37.488203,2013-03-07 09:46:46.0
133,서울특별시 강남구 개포동,12,12.0,0.0,삼익대청,51.12,202307,14,11,1992,개포로109길 21,NaN,20230817,중개거래,서울 강남구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9242,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산,66.96,202307,1,14,1995,신내로 155,NaN,20230915,중개거래,서울 중랑구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9243,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산,66.96,202307,27,6,1995,신내로 155,NaN,20230816,중개거래,"서울 송파구, 서울 중랑구",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9244,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산,84.99,202308,8,11,1995,신내로 155,NaN,,중개거래,서울 중랑구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9245,서울특별시 중랑구 신내동,656,656.0,0.0,화성.두산,84.99,202308,14,12,1995,신내로 155,NaN,20230908,직거래,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
test[~test['시군구'].isin(solution['시군구'])]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일


In [95]:
test[~test['전용면적(㎡)'].isin(solution['전용면적(㎡)'])]

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
2580,서울특별시 구로구 신도림동,643,643.0,0.0,동아1,169.199,202309,6,14,1999,신도림로 87,NaN,,중개거래,"서울 구로구, 서울 영등포구",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5855,서울특별시 서초구 서초동,1685-3,1685.0,3.0,아크로비스타,174.780,202308,11,37,2004,서초중앙로 188,NaN,,중개거래,서울 서초구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
test = pd.concat([test, gps], axis=1)
test = test.drop(['좌표X', '좌표Y'], axis=1)
test = test.rename(columns={'좌표X_2': '좌표X'})
test = test.rename(columns={'좌표Y_2': '좌표Y'})

In [97]:
test.sample(2)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,좌표X,좌표Y
7128,서울특별시 송파구 방이동,89,89.0,0.0,올림픽선수기자촌1단지,83.06,202307,6,9,1988,양재대로 1218,NaN,,중개거래,서울 송파구,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,127.136383,37.517573
5082,서울특별시 서대문구 홍제동,461,461.0,0.0,홍제삼성래미안,32.76,202309,9,8,2001,통일로 319,NaN,,중개거래,"경기 고양일산서구, 서울 서대문구",아파트,027301248,0263511248,NaN,분양,위탁관리,혼합식,개별난방,6.0,474.0,삼성래미안,삼성래미안,2001-06-13 00:00:00.0,56189.0,32649.0,42193.0,268.0,116.0,90.0,NaN,NaN,NaN,2023-09-26 08:40:21.0,90700167721,직영,단일계약,위탁,0.0,519.0,의무,2013-06-18 10:28:14.0,Y,N,2013-03-07 09:46:43.0,126.951312,37.578356


In [98]:
test['구'] = test['시군구'].apply(lambda x: x.split()[1])
test['동'] = test['시군구'].apply(lambda x: x.split()[2])

In [99]:
# 결측치 확인하고, 채워 넣을 피쳐 리스트 작성
missing_check_list = ['k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
              'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
              'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
              'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
              'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
              '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']

def missing_update(key_feature: str):
    # 결측치 채우기 전 결측치 수 및 비율 확인
    total_rows = len(test)
    missing_before = test[missing_check_list].isnull().sum().reset_index()
    missing_before.columns = ['Feature', 'Missing_Before']
    missing_before['Missing_Before_%'] = (missing_before['Missing_Before'] / total_rows) * 100

    # 결측치 채우기
    for feature in missing_check_list:
        if feature in add.columns:
            add_feature_dict = add.set_index(key_feature)[feature].to_dict()
            test[feature] = test.apply(lambda row: add_feature_dict.get(row[key_feature], row[feature]) if pd.isnull(row[feature]) else row[feature], axis=1)

    # 결측치 채우기 후 결측치 수 및 비율 확인
    missing_after = test[missing_check_list].isnull().sum().reset_index()
    missing_after.columns = ['Feature', 'Missing_After']
    missing_after['Missing_After_%'] = (missing_after['Missing_After'] / total_rows) * 100

    # 결측치 비교 테이블 생성
    missing_comparison = missing_before.merge(missing_after, on='Feature')

    # 결측치 비교 결과를 깔끔하게 출력
    display(missing_comparison)

In [100]:
# 새로운 피처 '동+아파트명' 생성
test['동+아파트명'] = test['동'] + ' ' + test['아파트명']
add['동+아파트명'] = add['주소(읍면동)'] + ' ' + add['k-아파트명']

missing_update('동+아파트명')

# key로 사용한 '동+아파트명' 피처는 삭제한다.
test.drop(columns=['동+아파트명'], inplace=True)

,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,6576,70.923210,6441,69.467213
1,k-팩스번호,6606,71.246764,6475,69.833909
2,단지소개기존clob,8718,94.025022,8682,93.636756
3,k-세대타입(분양형태),6562,70.772217,6427,69.316221
4,k-관리방식,6562,70.772217,6427,69.316221
5,k-복도유형,6564,70.793788,6429,69.337791
6,k-난방방식,6562,70.772217,6427,69.316221
7,k-전체동수,6577,70.933995,6442,69.477998
8,k-전체세대수,6562,70.772217,6425,69.294651
9,k-건설사(시공사),6579,70.955565,6447,69.531924


In [101]:
add['도로명'] = add['주소(도로명)'] + ' ' + add['주소(도로상세주소)']

missing_update('도로명')

,Feature,Missing_Before,Missing_Before_%,Missing_After,Missing_After_%
0,k-전화번호,6441,69.467213,1460,15.746333
1,k-팩스번호,6475,69.833909,1547,16.684642
2,단지소개기존clob,8682,93.636756,7698,83.024159
3,k-세대타입(분양형태),6427,69.316221,1467,15.821829
4,k-관리방식,6427,69.316221,1467,15.821829
5,k-복도유형,6429,69.337791,1467,15.821829
6,k-난방방식,6427,69.316221,1459,15.735548
7,k-전체동수,6442,69.477998,1467,15.821829
8,k-전체세대수,6425,69.294651,1457,15.713978
9,k-건설사(시공사),6447,69.531924,1489,16.059103


In [102]:
test['세대별주차대수'] = test['주차대수'] / test['k-전체세대수']

In [103]:
def impute_missing_values(df):
    # 수치형 피처와 범주형 피처 구분
    numeric_features = df.select_dtypes(include=[np.number]).columns
    categorical_features = df.select_dtypes(include=[object]).columns
    
    # 수치형 피처의 결측치를 평균값으로 대체
    for feature in numeric_features:
        mean_value = df[feature].mean()
        df[feature].fillna(mean_value, inplace=True)
    
    # 범주형 피처의 결측치를 'Missing'으로 대체
    for feature in categorical_features:
        df[feature].fillna('Missing', inplace=True)
    
    return df

impute_missing_values(test)

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,도로명,해제사유발생일,등기신청일자,거래유형,중개사소재지,"k-단지분류(아파트,주상복합등등)",k-전화번호,k-팩스번호,단지소개기존clob,k-세대타입(분양형태),k-관리방식,k-복도유형,k-난방방식,k-전체동수,k-전체세대수,k-건설사(시공사),k-시행사,k-사용검사일-사용승인일,k-연면적,k-주거전용면적,k-관리비부과면적,k-전용면적별세대현황(60㎡이하),k-전용면적별세대현황(60㎡~85㎡이하),k-85㎡~135㎡이하,k-135㎡초과,k-홈페이지,k-등록일자,k-수정일자,고용보험관리번호,경비비관리형태,세대전기계약방법,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,단지신청일,좌표X,좌표Y,구,동,세대별주차대수
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.9700,202307,26,5,1987,언주로 3,2.023085e+07,,직거래,-,아파트,025776611,025776673,504.012071,분양,자치관리,계단식,개별난방,8.0,270.0,우성건설,모름,1987-11-21 00:00:00.0,22637.0,20204.0,22637.0,20.0,250.0,0.0,70.0,Missing,2022-11-09 20:10:43.0,2023-09-23 17:21:41.0,Missing,직영,단일계약,직영,4858.00,262.0,임의,2022-11-17 13:00:29.0,Y,N,2022-11-17 10:19:06.0,127.056859,37.476276,강남구,개포동,0.970370
1,서울특별시 강남구 개포동,651-1,651.0,1.0,개포더샵트리에,108.2017,202308,15,10,2021,개포로 311,2.023085e+07,,중개거래,서울 강남구,아파트,025763999,025763998,504.012071,분양,위탁관리,계단식,지역난방,2.0,232.0,포스코,개포우성9차모델링조합,2021-12-01 00:00:00.0,44951.0,24878.0,31174.0,0.0,0.0,232.0,70.0,thesharp-trieh.hthomeservice.com/#/feeManage?_...,2022-01-05 18:02:23.0,2023-09-23 17:23:08.0,Missing,위탁,종합계약,위탁,2724.46,305.0,의무,2022-02-23 13:01:10.0,Y,N,2022-02-23 11:05:05.0,127.057257,37.484829,강남구,개포동,1.314655
2,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,161.0000,202307,28,15,1984,개포로 307,2.023085e+07,,중개거래,서울 강남구,아파트,025749116,025749094,2888.000000,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,70.0,Missing,Missing,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,127.056019,37.483975,강남구,개포동,1.034568
3,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,133.4600,202308,10,14,1984,개포로 307,2.023085e+07,,중개거래,서울 강남구,아파트,025749116,025749094,2888.000000,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,70.0,Missing,Missing,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,127.056019,37.483975,강남구,개포동,1.034568
4,서울특별시 강남구 개포동,652,652.0,0.0,개포우성3차,104.4300,202308,18,6,1984,개포로 307,2.023085e+07,,중개거래,서울 강남구,아파트,025749116,025749094,2888.000000,분양,자치관리,혼합식,지역난방,5.0,405.0,우성건설,우성건설,1984-12-31 00:00:00.0,63304.0,53437.0,61064.0,0.0,0.0,285.0,70.0,Missing,Missing,2023-09-23 17:15:43.0,21380015910,직영,단일계약,위탁,61064.24,419.0,의무,1984-12-22 00:00:00.0,Y,N,2013-03-07 09:46:28.0,127.056019,37.483975,강남구,개포동,1.034568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6500,202307,19,13,2014,신내역로1길 85,2.023080e+07,,직거래,-,아파트,024969683,024969685,4.000000,기타,위탁관리,혼합식,개별난방,16.0,1402.0,한신공영(주),SH공사,2014-06-18 00:00:00.0,190866.0,93849.0,92808.0,808.0,504.0,90.0,70.0,Missing,Missing,2023-09-23 09:00:52.0,2048279333,위탁,종합계약,위탁,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,2014-09-01 13:05:03.0,127.106467,37.617195,중랑구,신내동,1.118402
9268,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.6200,202307,25,12,2014,신내역로1길 85,2.023085e+07,,중개거래,서울 중랑구,아파트,024969683,024969685,4.000000,기타,위탁관리,혼합식,개별난방,16.0,1402.0,한신공영(주),SH공사,2014-06-18 00:00:00.0,190866.0,93849.0,92808.0,808.0,504.0,90.0,70.0,Missing,Missing,2023-09-23 09:00:52.0,2048279333,위탁,종합계약,위탁,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,2014-09-01 13:05:03.0,127.106467,37.617195,중랑구,신내동,1.118402
9269,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,101.6500,202308,27,12,2014,신내역로1길 85,2.023085e+07,,중개거래,서울 중랑구,아파트,024969683,024969685,4.000000,기타,위탁관리,혼합식,개별난방,16.0,1402.0,한신공영(주),SH공사,2014-06-18 00:00:00.0,190866.0,93849.0,92808.0,808.0,504.0,90.0,70.0,Missing,Missing,2023-09-23 09:00:52.0,2048279333,위탁,종합계약,위탁,14171.00,1568.0,의무,2015-09-09 15:30:27.0,Y,N,2014-09-01 13:05:03.0,127.106467,37.617195,중랑구,신내동,1.118402
9270,서울특별시 중랑구 신내동,816,816.0,0.0,신내우디안1단지,84.9400,202309,2,18,2

In [104]:
test.isna().mean() * 100

시군구                       0.0
번지                        0.0
본번                        0.0
부번                        0.0
아파트명                      0.0
전용면적(㎡)                   0.0
계약년월                      0.0
계약일                       0.0
층                         0.0
건축년도                      0.0
도로명                       0.0
해제사유발생일                   0.0
등기신청일자                    0.0
거래유형                      0.0
중개사소재지                    0.0
k-단지분류(아파트,주상복합등등)        0.0
k-전화번호                    0.0
k-팩스번호                    0.0
단지소개기존clob                0.0
k-세대타입(분양형태)              0.0
k-관리방식                    0.0
k-복도유형                    0.0
k-난방방식                    0.0
k-전체동수                    0.0
k-전체세대수                   0.0
k-건설사(시공사)                0.0
k-시행사                     0.0
k-사용검사일-사용승인일             0.0
k-연면적                     0.0
k-주거전용면적                  0.0
k-관리비부과면적                 0.0
k-전용면적별세대현황(60㎡이하)        0.0
k-전용면적별세대현황(60㎡~85㎡이하)    0.0
k-85㎡~135㎡

In [ ]:
# 파일 저장 부분이기 때문에, 실제 저장할 경우만 주석 풀고 사용!
test.to_csv('data/kkh_test.csv', index=False)